In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np 
from torchvision.datasets import MNIST
from scipy.spatial.distance import cdist
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd 

In [15]:
with open('./data/Bio_data/Gevers2014_meta.txt', 'r') as file:
    meta = file.read()

# Split the contents into lines
meta_lines = meta.split('\n')

# Remove empty lines and strip whitespace
meta_lines = [line.strip() for line in meta_lines if line.strip()]

# Split each line by tabs to create columns
meta_data = [line.split('\t') for line in meta_lines]

# Create a pandas DataFrame from the list of lists
meta_df = pd.DataFrame(meta_data)

# Count the number of rows and columns
num_rows, num_columns = meta_df.shape

# Print the counts
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Display the first few rows of the DataFrame for inspection
print(meta_df.head())

csv_file_path = './data/Bio_data/Gevers2014_meta.csv'

# Save the DataFrame as a CSV file
meta_df.to_csv(csv_file_path, index=False,header=0)

# Print a message to confirm that the file has been saved
print(f"DataFrame saved as {csv_file_path}")



Number of rows: 1360
Number of columns: 70
                   0                1                     2   \
0           #SampleID  BarcodeSequence  LinkerPrimerSequence   
1  SKBTI.1325.1246591             None   GTGCCAGCMGCCGCGGTAA   
2      121283.1246600             None   GTGCCAGCMGCCGCGGTAA   
3  SKBTI.0870.1246169             None   GTGCCAGCMGCCGCGGTAA   
4  SKBTI.1178.1246304             None   GTGCCAGCMGCCGCGGTAA   

                   3                  4                  5          6   \
0  TARGET_SUBFRAGMENT  ASSIGNED_FROM_GEO  EXPERIMENT_CENTER      TITLE   
1                  V4                  n                 BI  CCFA_RISK   
2                  V4                  n                 BI  CCFA_RISK   
3                  V4                  n                 BI  CCFA_RISK   
4                  V4                  n                 BI  CCFA_RISK   

                      7            8           9   ...                 60  \
0             RUN_PREFIX          AGE  MESALAMINE 

In [16]:
with open('./data/Bio_data/otutable.txt', 'r') as file:
    OTU = file.read()

# Split the contents into lines
OTU_lines = OTU.split('\n')

# Remove empty lines and strip whitespace
OTU_lines = [line.strip() for line in OTU_lines if line.strip()]

# Split each line by tabs to create columns
OTU_data = [line.split('\t') for line in OTU_lines]

# Create a pandas DataFrame from the list of lists
OTU_df = pd.DataFrame(OTU_data)

# Count the number of rows and columns
num_rows, num_columns = OTU_df.shape

# Print the counts
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Display the first few rows of the DataFrame for inspection
print(OTU_df.head())
csv_file_path = './data/Bio_data/Gevers2014_OTU.csv'

# Save the DataFrame as a CSV file
OTU_df.to_csv(csv_file_path, index=False,header=0)

# Print a message to confirm that the file has been saved
print(f"DataFrame saved as {csv_file_path}")

Number of rows: 944
Number of columns: 1359
                                                0               1     \
0                                            #OTU ID  100052.1246453   
1  NR_112946.1_Bacteroides_vulgatus_strain_JCM_58...            1373   
2  NR_041351.1_Bacteroides_dorei_strain_175_16S_r...              17   
3  NR_041342.1_Parabacteroides_distasonis_strain_...               0   
4  NR_025930.1_Ruminococcus_bromii_strain_ATCC_27...               0   

             2               3               4               5     \
0  100216.1246217  100211.1246832  100083.1246311  100095.1246168   
1            2669               0             311               1   
2            1993               1             165             623   
3             814               0               0              34   
4               0               1               0               0   

             6               7               8               9     ...  \
0  100043.1246460  100212.1247085 

In [23]:
# Define the path to the CSV file
csv_file_path = './data/Bio_data/Gevers2014_meta.csv'
csv_file_path2 = './data/Bio_data/Gevers2014_OTU.csv'

# Read the CSV file into a DataFrame
meta_df = pd.read_csv(csv_file_path)
OTU_df = pd.read_csv(csv_file_path2)


In [20]:
# Read the CSV file into a DataFrame
meta_df = pd.read_csv(csv_file_path)

# Use value_counts() to count unique values in the 'TARGET_SUBFRAGMENT' column
value_counts = meta_df['TARGET_SUBFRAGMENT'].value_counts()

# Display the unique values and their counts
print(value_counts)

V4    1359
Name: TARGET_SUBFRAGMENT, dtype: int64


In [25]:
meta_df

,#SampleID,BarcodeSequence,LinkerPrimerSequence,TARGET_SUBFRAGMENT,ASSIGNED_FROM_GEO,EXPERIMENT_CENTER,TITLE,RUN_PREFIX,AGE,MESALAMINE,...,ILEAL_INVOVLEMENT,REGION,RUN_CENTER,PCR_PRIMERS,LIBRARY_CONSTRUCTION_PROTOCOL,GASTROINTEST_DISORD,LATITUDE,EXTERNAL_ID,Description_duplicate,Description
0,SKBTI.1325.1246591,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A2G81.1.Solexa-161197,9.083333333,no,...,yes,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,IC,42.358,SKBTI.1325,M1-0741 biopsy,CCFA_RISK
1,121283.1246600,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A2NWB.1.Solexa-136332,22.0,None,...,yes,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,CD,42.358,121283,8603 stool,CCFA_RISK
2,SKBTI.0870.1246169,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A29H3.1.Solexa-146974,15.41666667,no,...,no,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,CD,42.358,SKBTI.0870,M1-0606 biopsy,CCFA_RISK
3,SKBTI.1178.1246304,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A4848.1.Solexa-162352,12.16666667,no,...,None,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,no,42.358,SKBTI.1178,M1-0453 biopsy,CCFA_RISK
4,MGH101468.1246593,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A2246.1.Solexa-125757,49.0,yes,...,no,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,CD,42.358,MGH101468,7072 biopsy,CCFA_RISK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1354,SKBTI.0518.1246864,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A1UEN.1.Solexa-133653,13.75,no,...,yes,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,CD,42.358,SKBTI.0518,M1-0506 biopsy,CCFA_RISK
1355,SKBTI.1317.1247132,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A2G81.1.Solexa-161192,9.416666667,no,...,None,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,CD,42.358,SKBTI.1317,M1-0810 biopsy,CCFA_RISK
1356,SKBTI024.1246988,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A2WP4.1.Solexa-106895,16.25,no,...,no,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,CD,42.358,SKBTI024,M1-0072 biopsy,CCFA_RISK
1357,SKBTI.1035.1246669,None,GTGCCAGCMGCCGCGGTAA,V4,n,BI,CCFA_RISK,A2G81.1.Solexa-161153,11.08333333,no,...,yes,None,BI,FWD:GTGCCAGCMGCCGCGGTAA; REV:GGACTACHVGGGTWTCTAAT,This analysis was done as in Caporaso et al 20...,CD,42.358,SKBTI.1035,M1-0821 biopsy,CCFA_RISK


In [24]:
OTU_df








,#OTU ID,100052.1246453,100216.1246217,100211.1246832,100083.1246311,100095.1246168,100043.1246460,100212.1247085,100206.1246938,122057.1246885,...,SKBTI.0767.1246917,121180.1246978,MGH106271.1246160,SKBTI.0652.1246739,SKBTI.0593.1247128,SKBTI.0914.1246971,SKBTI.0448.1246164,SKBTI.0939.1246784,SKBTI.0692.1246233,SKBTI.0362.1246358
0,NR_112946.1_Bacteroides_vulgatus_strain_JCM_58...,1373,2669,0,311,1,3,5,0,2,...,0,0,0,0,0,0,0,0,0,0
1,NR_041351.1_Bacteroides_dorei_strain_175_16S_r...,17,1993,1,165,623,1636,2500,0,144,...,0,0,0,0,0,0,0,0,0,0
2,NR_041342.1_Parabacteroides_distasonis_strain_...,0,814,0,0,34,0,0,0,16,...,0,0,0,0,0,0,0,0,0,0
3,NR_025930.1_Ruminococcus_bromii_strain_ATCC_27...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NR_025651.1_{Clostridium}_lactatifermentans_st...,68,122,1,1,102,4,0,0,8,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,NR_147406.1_Mordavella_massiliensis_strain_Mar...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
939,NR_147712.1_Massilia_putida_strain_6NM-7_16S_r...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
940,NR_147709.1_Lactobacillus_wasatchensis_strain_...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
941,NR_148266.1_Clostridium_vulturis_strain_YMB-57...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
